In [1]:
from openai import OpenAI
import json
import re
from datasets import load_dataset
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

key = os.getenv("API_KEY")
client = OpenAI(api_key=key, base_url="https://api.deepseek.com")

In [3]:
ner_dict = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [4]:
dataset = load_dataset("eriktks/conll2003")
#print(dataset['train']['id'])
#print(dataset['train']['tokens'])
#print(dataset['train']['pos_tags'])
#print(dataset['train']['chunk_tags'])
#print(dataset['train']['ner_tags'][:15])

In [11]:
# Evaluation of results

def parse_prediction(pred_str, ner_dict):
    pred_str = pred_str.strip()
    if pred_str.startswith("["):
        pred_str = pred_str[1:]
    if pred_str.endswith("]"):
        pred_str = pred_str[:-1]
    items = pred_str.split("),")
    result = []
    for item in items:
        item = item.replace("(", "").replace(")", "").replace("'", "").replace(",", "")
        parts = item.strip().split()
        if len(parts) == 2:
            token, label = parts
            result.append(ner_dict.get(label, 0))
    return result

def f1_score(file_path, dataset, dict=ner_dict):
    """
    Calculate the F1 score for the predictions in the file.
    """
    with open(file_path, 'r') as f:
        predictions = f.readlines()

    predictions = [parse_prediction(line.strip(), dict) for line in predictions]

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0
    for i, pred in enumerate(predictions):
        true_labels = dataset['train']['ner_tags'][i]
        if len(pred) != len(true_labels):
            print(f"Warning: Length mismatch at index {i}. Predicted: {len(pred)}, True: {len(true_labels)}\n")
            print(f"True labels: {true_labels},\n Predicte--: {pred}")
            return
        for j, label in enumerate(pred):
            if label == true_labels[j]:
                true_positives += 1
            else:
                false_positives += 1
                false_negatives += 1

f1_score("vanilla.txt", dataset, ner_dict)


True labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0],
 Predicte--: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 1, 2, 2, 2, 0, 0, 0, 0, 0]


In [5]:
def get_ner_label(value, ner_dict):
    for k, v in ner_dict.items():
        if v == value:
            return k
    return None

In [6]:
print(dataset['train']['tokens'][0])
print([get_ner_label(x, ner_dict) for x in dataset['train']['ner_tags'][0]])

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [7]:
def send_prompt(prompt):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            #{"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )

    return response.choices[0].message.content

### VANILLA METHOD

In [8]:
for sentence in dataset['train']['tokens']:
    prompt = f"Given the entity label set: {list(ner_dict.keys())}.\n \
        Based on the given entity label set, please recognize the named entities in the given text.\n \
        Return only a list of tuples with each token and its label, nothing else. For example [('In','O'), ('America','I-LOC'), ('is','O'), ('cold','O'), ...]\n \
        Text: {" ".join(sentence)}"

    answer = send_prompt(prompt)
    # print(answer)
    # tags = re.findall(r"\('.*?',\s*'(.*?)'\)", answer)
    # tags = [ner_dict[tag] for tag in tags]
    # print(tags)
    with open("vanilla.txt", "a") as file:
        file.write(f"{answer}\n")


KeyboardInterrupt: 

### Q&A